<a href="https://colab.research.google.com/github/thisisquangphuc/fhdo-distributed-system/blob/issac/TruckPlatooning_CPP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Install the NVIDIA CUDA Compile
!apt-get update
!apt-get install -y nvidia-cuda-toolkit


In [ ]:
#This code implements GPU-accelerated distance calculations and CPU-based obstacle avoidance
# It uses CUDA and C++ multithreading.


#to save program in .cu file

%%writefile truck_platooning.cu

#include <iostream>
#include <vector>
#include <cmath>
#include <thread>
#include <mutex>
#include <cuda_runtime.h>
#include <iomanip> // For formatting distances

// Constants
const float DEFAULT_SAFETY_DISTANCE = 30.0; // Default safety distance
const float AVOIDANCE_FACTOR = 1.5;         // Factor for avoidance adjustment
const int NUM_THREADS = 2;                  // Number of threads
const float BRAKING_DISTANCE = 15.0;        // Distance at which braking is applied

// Function to calculate distances between trucks
std::vector<std::vector<float>> calculate_truck_distances(const std::vector<std::vector<float>> &truck_positions) {
    int num_trucks = truck_positions.size();
    std::vector<std::vector<float>> distances(num_trucks, std::vector<float>(num_trucks, 0.0));

    for (int i = 0; i < num_trucks; ++i) {
        for (int j = i + 1; j < num_trucks; ++j) {
            float dx = truck_positions[i][0] - truck_positions[j][0];
            float dy = truck_positions[i][1] - truck_positions[j][1];
            distances[i][j] = distances[j][i] = sqrtf(dx * dx + dy * dy);
        }
    }
    return distances;
}

// Function for obstacle detection and avoidance
void obstacle_detection_and_avoidance(std::vector<std::vector<float>> &truck_positions,
                                      const std::vector<std::vector<float>> &obstacle_positions,
                                      std::vector<std::vector<int>> &logical_clock,
                                      int thread_id,
                                      std::vector<std::string> &avoidance_methods) {
    int num_trucks = truck_positions.size();

    for (int i = 0; i < num_trucks; ++i) {
        float min_distance = DEFAULT_SAFETY_DISTANCE + 1;
        int closest_obstacle = -1;

        // Find the closest obstacle
        for (size_t j = 0; j < obstacle_positions.size(); ++j) {
            float dx = truck_positions[i][0] - obstacle_positions[j][0];
            float dy = truck_positions[i][1] - obstacle_positions[j][1];
            float distance = sqrtf(dx * dx + dy * dy);

            if (distance < min_distance) {
                min_distance = distance;
                closest_obstacle = j;
            }
        }

        // If too close, apply braking or avoidance logic
        if (min_distance < DEFAULT_SAFETY_DISTANCE && closest_obstacle != -1) {
            if (min_distance < BRAKING_DISTANCE) {
                // Apply braking: truck slows down
                truck_positions[i][0] -= 2.0; // Simulating braking
                truck_positions[i][1] -= 2.0;
                avoidance_methods[i] = "Braking";
            } else {
                // Adjust position to avoid obstacle
                truck_positions[i][0] += AVOIDANCE_FACTOR * (truck_positions[i][0] - obstacle_positions[closest_obstacle][0]);
                truck_positions[i][1] += AVOIDANCE_FACTOR * (truck_positions[i][1] - obstacle_positions[closest_obstacle][1]);
                avoidance_methods[i] = "Avoidance Maneuver";
            }
        } else {
            avoidance_methods[i] = "No Avoidance Needed";
        }

        // Update logical clock
        logical_clock[thread_id][thread_id]++;
    }
}

// Main simulation
int main() {
    // Simulate 3 trucks in a platoon and 1 obstacle
    std::vector<std::vector<float>> truck_positions = {{50.0, 50.0}, {60.0, 60.0}, {70.0, 70.0}};
    std::vector<std::vector<float>> obstacle_positions = {{65.0, 65.0}};
    std::vector<std::vector<int>> logical_clock(NUM_THREADS, std::vector<int>(NUM_THREADS, 0));

    // Output initial positions
    std::cout << "Initial Truck Positions:\n";
    for (const auto &pos : truck_positions) {
        std::cout << pos[0] << ", " << pos[1] << "\n";
    }

    std::cout << "\nObstacle Position:\n";
    for (const auto &pos : obstacle_positions) {
        std::cout << pos[0] << ", " << pos[1] << "\n";
    }

    // Calculate and display initial distances between trucks
    std::cout << "\nInitial Distances Between Trucks:\n";
    auto truck_distances = calculate_truck_distances(truck_positions);
    for (const auto &row : truck_distances) {
        for (const auto &distance : row) {
            std::cout << std::setw(8) << std::fixed << std::setprecision(2) << distance;
        }
        std::cout << "\n";
    }

    // Multithreading for processing
    std::vector<std::thread> threads;
    std::mutex mutex;

    int chunk_size = (truck_positions.size() + NUM_THREADS - 1) / NUM_THREADS; // Ensure proper chunking
    std::vector<std::string> avoidance_methods(truck_positions.size(), "");

    for (int t = 0; t < NUM_THREADS; ++t) {
        int start_idx = t * chunk_size;
        int end_idx = std::min(static_cast<int>(truck_positions.size()), start_idx + chunk_size);

        threads.emplace_back([&, t, start_idx, end_idx]() {
            std::vector<std::vector<float>> truck_chunk(truck_positions.begin() + start_idx, truck_positions.begin() + end_idx);
            obstacle_detection_and_avoidance(truck_chunk, obstacle_positions, logical_clock, t, avoidance_methods);

            std::lock_guard<std::mutex> lock(mutex);
            std::copy(truck_chunk.begin(), truck_chunk.end(), truck_positions.begin() + start_idx);
        });
    }

    for (auto &thread : threads) {
        thread.join();
    }

    // Display avoidance methods
    std::cout << "\nAvoidance Methods for Trucks:\n";
    for (int i = 0; i < avoidance_methods.size(); ++i) {
        std::cout << "Truck " << i + 1 << ": " << avoidance_methods[i] << "\n";
    }

    // Display final truck positions
    std::cout << "\nFinal Truck Positions After Avoidance:\n";
    for (const auto &pos : truck_positions) {
        std::cout << pos[0] << ", " << pos[1] << "\n";
    }

    // Display updated logical clock matrix
    std::cout << "\nLogical Clock Matrix:\n";
    for (const auto &row : logical_clock) {
        for (const auto &val : row) {
            std::cout << std::setw(5) << val;
        }
        std::cout << "\n";
    }

    return 0;
}


Overwriting truck_platooning.cu


In [ ]:
# Compile the code using the NVIDIA compiler:

!nvcc truck_platooning.cu -o truck_platooning


In [ ]:
#Run the compiled program

!./truck_platooning


Updated Truck Positions (first 10):
411.5, 929.5
771, 898.5
793, 335
315.5, 477
619, 442
362, 27
685.5, 92
736, 926
540, 426
172, 736
